<a href="https://colab.research.google.com/github/FilipeFilpe/puc-mvp2-colab/blob/main/ESSI_MVP_PREDICAO_COLUNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Engenharia de Sistemas de Software Inteligentes - Profs. Marcos Kalinowski e Tatiana Escovedo
## MVP - Criação de um modelo de predição

## Objetivo
Prever se determinado paciente ortopédico tem algum problema de coluna (hérnia de disco ou espondilolistese) ou se a coluna está normal.

Link do dataset original https://archive.ics.uci.edu/dataset/212/vertebral+column



In [ ]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

## Carga do Dataset

In [ ]:
# Informa a URL de importação do dataset
url = "https://gist.githubusercontent.com/FilipeFilpe/e8fa5d23d6fce79f435759c76fe4665d/raw/cc6a3af7623341fbc1aa74f61eddbc9f41880905/vertebral-column.csv"

# Lê o arquivo
dataset = pd.read_csv(url, delimiter=',')

# Mostra as primeiras linhas do dataset
dataset.head()

,pelvic incidence,pelvic tilt,lumbar lordosis angle,sacral slope,pelvic radius,grade of spondylolisthesis,result
0,63.03,22.55,39.61,40.48,98.67,-0.25,AB
1,39.06,10.06,25.02,29.00,114.41,4.56,AB
2,68.83,22.22,50.09,46.61,105.99,-3.53,AB
3,69.30,24.65,44.31,44.64,101.87,11.21,AB
4,49.71,9.65,28.32,40.06,108.17,7.92,AB


## Separação em conjunto de treino e conjunto de teste com holdout

In [ ]:
test_size = 0.20 # tamanho do conjunto de teste
seed = 7 # semente aleatória

# Separação em conjuntos de treino e teste
array = dataset.values
X = array[:,0:6]
y = array[:,6]
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=test_size, shuffle=True, random_state=seed, stratify=y) # holdout com estratificação

# Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 20
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # validação cruzada com estratificação

## Modelagem e Inferência

Instanciação das classes utilizadas e criação do pipeline.

In [ ]:
np.random.seed(7) # definindo uma semente global para este bloco

# Listas para armazenar os armazenar os pipelines
pipelines = []

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())

# Algoritmos que serão utilizados
knn = ('KNN', KNeighborsClassifier())
cart = ('CART', DecisionTreeClassifier())
naive_bayes = ('NB', GaussianNB())
svm = ('SVM', SVC())

# Montando os pipelines
# Dataset original
pipelines.append(('KNN-orig', Pipeline([knn])))
pipelines.append(('CART-orig', Pipeline([cart])))
pipelines.append(('NB-orig', Pipeline([naive_bayes])))
pipelines.append(('SVM-orig', Pipeline([svm])))

# Dataset Padronizado
pipelines.append(('KNN-padr', Pipeline([standard_scaler, knn])))
pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
pipelines.append(('NB-padr', Pipeline([standard_scaler, naive_bayes])))
pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))

# Dataset Normalizado
pipelines.append(('KNN-norm', Pipeline([min_max_scaler, knn])))
pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
pipelines.append(('NB-norm', Pipeline([min_max_scaler, naive_bayes])))
pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))


### Criação e avaliação de modelos: dados padronizados e normalizados

Comparação das avaliações e escolha do melhor resultado padronizado e normalizado.

In [ ]:
# Guarda o melhor resultado
best_result_padr_norm = ('', 0)

# Executando os pipelines
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)

    if (best_result_padr_norm[1] < cv_results.mean()):
      best_result_padr_norm = (name, cv_results.mean())

print('BEST RESULT', best_result_padr_norm)

BEST RESULT ('SVM-padr', 0.8583333333333332)


### Otimização dos hiperparâmetros

Avaliação e comparação para escolher o melhor resultado com hiperparâmetros.

In [ ]:
# Tuning do KNN, CART, NB e SVM

# Guarda o melhor resultado
best_result_hypers = ('', 0, '')

# KNN
param_grid_knn = {
    'KNN__n_neighbors': [1,3,5,7,9,11,13,15,17,19,21],
    'KNN__metric': ["euclidean", "manhattan", "minkowski"]
}

# CART
param_grid_cart = {
    'CART__max_depth':[3,5,7,10,15],
    'CART__min_samples_leaf':[3,5,10,15,20],
    'CART__min_samples_split':[8,10,12,18,20,16],
    'CART__criterion':['gini','entropy']
}

# NB
param_grid_nb = {
    'NB__var_smoothing':[1e+4,1e-9,3e-10,5e-10]
}

# SVM
param_grid_svm = {
    'SVM__C': [0.1,1, 10, 100],
    'SVM__gamma':[1,0.1,0.01,0.001]
}

param_grid = {'KNN': param_grid_knn, 'CART': param_grid_cart, 'NB': param_grid_nb, 'SVM': param_grid_svm}

# Prepara e executa o GridSearchCV
for name, model in pipelines:
    grid = GridSearchCV(estimator=model, param_grid=param_grid[name.split('-')[0]], scoring=scoring, cv=kfold)
    grid.fit(X_train, y_train)

    if (best_result_hypers[1] < grid.best_score_):
      best_result_hypers = (name, grid.best_score_, grid.best_params_)

print('BEST RESULT', best_result_hypers)

BEST RESULT ('SVM-orig', 0.8708333333333333, {'SVM__C': 10, 'SVM__gamma': 0.001})


In [ ]:
# Melhor configuração para a criaçao do modelo
if best_result_hypers > best_result_padr_norm:
  best_results = best_result_hypers
else:
  best_results = best_result_padr_norm

print('BEST RESULT', best_results)


BEST RESULT ('SVM-padr', 0.8583333333333332)


## Finalização do Modelo

In [ ]:
# Avaliação do modelo com o conjunto de testes

# Preparação do modelo
scaler = StandardScaler().fit(X_train) # ajuste do scaler com o conjunto de treino
rescaledX = scaler.transform(X_train) # aplicação da padronização no conjunto de treino
model = SVC(C=10, gamma=0.001)
model.fit(rescaledX, y_train)

# Estimativa da acurácia no conjunto de teste
rescaledTestX = scaler.transform(X_test) # aplicação da padronização no conjunto de teste
predictions = model.predict(rescaledTestX)
print(accuracy_score(y_test, predictions))

0.8225806451612904


In [ ]:
# Preparação do modelo com TODO o dataset
scaler = StandardScaler().fit(X) # ajuste do scaler com TODO o dataset
rescaledX = scaler.transform(X) # aplicação da padronização com TODO o dataset
model.fit(rescaledX, y)

SVC(C=10, gamma=0.001)

## Simulando a aplicação do modelo em dados não vistos

In [ ]:
# Novos dados - não sabemos a classe!
data = {
    'pelvic incidence': [68.83,39.66,91.47,48.9],
    'pelvic tilt': [22.22,16.21,24.51,5.59],
    'lumbar lordosis angle': [50.09,36.67,84.62,55.5],
    'sacral slope': [46.61,23.45,66.96,43.32],
    'pelvic radius': [105.99,131.92,117.31,137.11],
    'grade of spondylolisthesis': [-3.53,-4.97,52.62,19.85],
}

atributos = ['pelvic incidence', 'pelvic tilt', 'lumbar lordosis angle', 'sacral slope', 'pelvic radius', 'grade of spondylolisthesis']
entrada = pd.DataFrame(data, columns=atributos)

array_entrada = entrada.values
X_entrada = array_entrada[:,:].astype(float)

# Padronização nos dados de entrada usando o scaler utilizado em X
rescaledEntradaX = scaler.transform(X_entrada)
print(rescaledEntradaX)

[[ 0.48427345  0.46808485 -0.0993699   0.27282344 -0.89729467 -0.79541679]
 [-1.21084014 -0.13339724 -0.82384264 -1.45539534  1.05289655 -0.83381858]
 [ 1.79991868  0.69726855  1.76471683  1.79135764 -0.04591925  0.70198621]
 [-0.67388952 -1.19624913  0.19268655  0.02732086  1.44323564 -0.17192112]]


In [ ]:
# Predição de classes dos dados de entrada
saidas = model.predict(rescaledEntradaX)
print(saidas) # RESULTADO ESPERADO ['AB' 'NO' 'AB' 'NO']

['AB' 'NO' 'AB' 'NO']


## Exportação do modelo

In [ ]:
# Criação de um arquivo para exportar o modelo

pickle_out = open('modelo_classificador.pkl', 'wb')
pickle.dump(model, pickle_out)
pickle_out.close()

['AB' 'NO' 'AB' 'NO']


## Análise de resultados do modelo


Apesar do dataset não ser muito grande, ele se mostrou ser o suficiente para o treinamento do modelo mostrando-se muito eficiente.

Muito provavelmente se o dataset fosse maior, o desempenho do modelo poderia ser muito melhor.
